# Set data directory

In [12]:
data_dir = '2021_10_06_test_data'

# make switch between 'upper' and 'lower' to match display
plt.rc('image',origin='upper')

In [9]:
# grab all the fits files
files = glob.glob(os.path.join(data_dir,'*.FIT'))

In [ ]:
# List the object in each file
for f in files:
    with fits.open(f) as hdu:
        print('{:<80s}'.format(f),hdu[0].header['OBJECT'])

In [ ]:
# make a little gallery of object
fig, axs = plt.subplots(int(len(files)/4 + 1),4,figsize=(13,13))
for i,f in enumerate(files):
    with fits.open(f) as hdu:
        axs.flat[i].imshow(hdu[0].data,cmap='turbo',norm=mpl.colors.LogNorm(vmin=100,vmax=65000/2))
        axs.flat[i].set_title(os.path.basename(f)[:-4])

In [ ]:
# Get all of our helper functions
%run helper_funcs
%run plot_and_reduce



# reads in functions
- Utilities
    - `find_peaks(arr,threshold = 0.1 , size=20,axis=-1)`
        - finds peaks in arrays
    - `get_cosmic_rays(data)`
        - zaps cosmic rays in images with weak lines
    - `shift_row_interp(A, wave_poly,plot=True,fig=None,axs=None)`
        - correct row position offsets. offsets are describes by polynomial `wave_poly`
- Data reduction
    - `wavelength_cal(peaks,hg,ar,order=1)`
        - derive the wavelength calibration using peaks from `find_peaks`
    - `specextract(data,bottom=None,top=None,plot=True,fig=None,ax=None)`
        - extract regions containing spectrum. if top & bottom are not provided, it will attempt to determine them
        - Automatic line finding only works for stars
    - `rectify_ccd(cal, order=1, plot=True,fig=None,ax=None)`
        - rectify (fix pixel offsets) for the data. output of `rectify` is `wave_poly` and an array with the original offset pixel coordinates
    - `get_wavelength_solution(cal,threshold=0.05,size=5,plot=False)`
        - quickly get a wavelength solution given a rectified or narrow slice of calibration image
    - `reduce(cal_file, data_file, cal_threshold=0.05, bottom = None, top = None,plot=True,clip_cal=False,cosmic_rays=False)`
        - Full reduce the data_file using cal_file for wavelength solution. 



In general one should use the `reduce` function and adjust top,bottom, threshold, to get a good wavelength calibration

# Automatic Data Reduction

In [ ]:
# put in your own files
cal_file = data_dir+'/calibration_3_20sec.FIT'
data_file = data_dir+'/alcyone_150sec.FIT'

# plot = True will show all diagnostic plots
# plot = False will only show wavelength calibration and final spectrum
out = reduce(cal,data,rectify=True,plot=True)


Below you can walk through each step of the data reduction pipeline. 

This exposes more parameters and intermediate outputs.

This should run with out needing to change anything.

# 1) Read in data

1) Read in your `calibration` and `data` files, and line list

In [ ]:
# set to true if you want to plot
plot = False
# load line list and convert to nanometers
hg = np.loadtxt('hgar_blue.txt') / 10
ar = np.loadtxt('argon_red.txt') / 10 

In [ ]:
# these need to be floats
data = fits.getdata(data_file).astype(float)
cal = fits.getdata(cal_file).astype(float)

if plot:
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(11,4))

    norm = mpl.colors.LogNorm(vmin=50,vmax=10000)
    ax1.imshow(cal,origin='upper',norm=norm,cmap='magma')
    ax1.set_title('Calibration')
    ax2.imshow(data,origin='upper',norm=norm,cmap='magma')
    ax2.set_title('Data')

# 2) Rectify the images

In [ ]:
oldcal = cal.copy() # save a copy so we can rerun this cell and show it

In [ ]:
# Rectify the data

# 1) solve the image plane
rect_sol, full_frame_solution = rectify_ccd(oldcal,order=1,plot=False)
# 2) apply the found solution
cal = shift_row_interp(cal,rect_sol,plot=False)
data = shift_row_interp(data,rect_sol,plot=False)

if plot:
    norm = mpl.colors.LogNorm(vmin=50,vmax=10000)
    plt.imshow(oldcal,origin='upper',cmap='magma',norm=norm)
    plt.contour(full_frame_solution,colors=['w']) # overlay the unrectified grid


    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(11,4))


    ax1.imshow(cal,origin='upper',norm=norm,cmap='magma')
    ax1.contour(shift_row_interp(full_frame_solution,rect_sol,plot=False),colors='w')
    ax1.set_title('Calibration - rectified')
    ax2.imshow(data,origin='upper',norm=norm,cmap='magma')
    ax2.set_title('Data - rectified')


# 3) Get spectral range

In [ ]:
if plot:
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(10,3))
else:
    ax1, ax2 = None,None


# try the manually by inspecting
# 'Data-rectified' figure (or just put plt.imshow(data) in a new cell)

# specextract has it's own complex plotting routine
sl = specextract(data,bottom=100,top=200,plot=plot,ax=ax2)


# try the automated way
sl = specextract(data,plot=plot,ax=ax1)

if plot:
    ax2.set_title('manual')
    ax1.set_title('automatic')


# 4) Wavelength calibration

In [ ]:
# load line list and convert to nanometers
hg = np.loadtxt('hgar_blue.txt') / 10
ar = np.loadtxt('argon_red.txt') / 10 

In [ ]:
# We can run the functions ourselves
cal_spec = np.nanmean(cal[sl,:],axis=0)
peaks = find_peaks(cal_spec,threshold=.05,size=5,)[0][::-1] # put in wavelength order
if plot:
    plt.plot(cal_spec)
    plt.plot(peaks,cal_spec[peaks],'r.')

p = wavelength_cal(peaks,hg,ar)
λ = np.polyval(p,np.arange(len(cal_spec)))

# or we can use the convenience function
p, λ, cal_spec = get_wavelength_solution(cal[sl,:])

# 5) Plot target spectrum

In [ ]:
# the 100 accounts for the 100 valued bias/offset
spec = np.mean(data[sl,:] -97., axis = 0)
noise = np.sqrt(spec)

plt.errorbar(λ, spec,noise)

# 6) Save the spectrum

In [ ]:
out = list(zip(λ,spec,noise))
fname = f"{data_file.replace('.FIT','.tsv')}"

In [ ]:
with open(fname,'w') as f:
    f.write('wavelength\tspectrum\terror\n')
    for i in out:
        f.write('{:<9.3f}\t{:>10.3f}\t{:>10.3f}\n'.format(*i))

In [ ]:
plt.plot(wavesol_planet,(spec_planet),label='Saturn')
plt.plot(wavesol_ring,(spec_ring),label='Ring')
plt.legend()
plt.xlabel('Wavelength [nm]')
plt.ylabel('Scaled data units')

In [ ]:
s = np.ptp(spec_jup)
ju.errorbar_fill(wavesol_jup,ju.scale_ptp(spec_jup),noise_jup/s,zorder=0,alpha=0.7)
s = np.ptp(spec_io)
ju.errorbar_fill(wavesol_io,ju.scale_ptp(spec_io),noise_io/s,zorder=1,alpha=0.7)

In [ ]:
cal = data_dir+'/calibration_3_20sec.FIT'
data = data_dir+'/orion_nebulosity_150sec.FIT'

λ,spec,err = reduce(cal,data,bottom=0,top=510,rectify=True,plot=False,)

data = data_dir+'/alcyone_150sec.FIT'
λ,spec,err = reduce(cal,data,rectify=True,plot=False,)


In [ ]:
def findback1d(image,s=31,fill=0,experimental=False):
    image = np.copy(image)
    oldnan = np.isnan(image)
    
    sp = s + s//4
    sp = sp+1 if sp%2==0 else sp
    s1 = sp//2
    s1 = s1+1 if s1%2==0 else s1
    s2 = s1//2
    arr=np.ones((sp,))
    arr[s1-s2:s1+s2]=0
    expand =convolve_fft(image,kernels.CustomKernel(arr),boundary='wrap')
    image[np.isnan(image)] = expand[np.isnan(image)]
    
    #image[oldnan] = fill
    s = int(s)
    s = s+1 if s%2==0 else s
    bkg_min = ndimage.minimum_filter(image,size=(s,))
    bkg_max = ndimage.maximum_filter(bkg_min,size=(s,))
    kernel = kernels.Box1DKernel(s)
    bkg_mean = convolve(bkg_max,kernel,boundary='extend',)
    
    
    if experimental:
        bkg_mean = np.min(np.vstack([[bkg_max],[bkg_mean]]),axis=0)
        bkg_new = np.copy(bkg_mean)
        #print(bkg_new.shape)
        s=s//2
        while s>2:
            s = s+1 if s%2==0 else s
            kernel2 = kernels.Box1DKernel(s)
            bkg_mean = convolve_fft(bkg_new,kernel2,boundary='wrap')
            bkg_new = np.min(np.array([bkg_mean,bkg_new]),axis=0)
            s=s//2

        bkg_new = np.min(np.vstack([[bkg_mean],[bkg_new]]),axis=0)
        kernel3 = kernels.CustomKernel(np.ones((1,)))
        kernel3.normalize()
        bkg_mean = convolve_fft(bkg_new,kernel,boundary='wrap')
    
    
    bkg = bkg_mean
    
    
    bkg[oldnan] = np.nan

    return bkg




In [ ]:
def get_wavelength_solution(cal,threshold=0.05,size=5,plot=False):
    if isinstance(cal,str):
        cal = fits.getdata(cal)
        
    wavesol, _ = rectify_ccd(cal,plot=plot)
    rect = shift_row_interp(cal,wavesol,plot=plot)
    cal_spec = np.nanmean(rect,axis=0)
    peaks = find_peaks(cal_spec,threshold=threshold,size=size,)[0][::-1] # put in wavelength order
    p = wavelength_cal(peaks,hg,ar)
    λ = np.polyval(p,np.arange(rect.shape[1]))
    return p, λ, rect

In [ ]:
back = findback1d(np.mean(rect,axis=0),s=50,experimental=False)

In [ ]:
tellurics = np.loadtxt('atmabs.txt',skiprows=4)

In [ ]:
kde=stats.gaussian_kde(tellurics[:,3]/10,weights=1-tellurics[:,2],bw_method=.007)
tell_x = np.linspace(200,1000,10000)

In [ ]:
plt.figure(figsize=(9,3),facecolor='w')
plt.plot(λ,np.mean(rect-np.median(rect),axis=0),'k',zorder=1,label='Calibration Spectrum')
plt.fill_between(tell_x,400*ju.scale_ptp(kde(tell_x)),lw=1,zorder=2,color='C0',alpha=0.5,label='Tellurics')
#plt.bar(tellurics[:,3]/10,1-tellurics[:,2],width=(tellurics[:,1]-tellurics[:,0])/10)
# for t in tellurics:
#     plt.axvspan(t[0]/10,t[1]/10,t[2]+.1,1,color='C4',zorder=0)
for c in hg:
    plt.axvline(c,0,.075,color='dodgerblue',lw=2,zorder=0)
for c in ar:
    plt.axvline(c,0,.075,color='indianred',lw=2,zorder=0)
#plt.xlim(650,750)
plt.xlim(500,800)
plt.ylim(-20,300)
plt.xlabel('Wavelength [nm]')
plt.legend(loc='upper left')
plt.savefig('Calibation_spectrum_with_tellurics.png',transparent=False)